In [1]:
import wave
import numpy as np
from scipy.io import wavfile

In [2]:
audioPath = "audio.wav"
msg = input("enter")
msg += "$"
msgInBinary = ''.join([format(ord(i), "08b") for i in msg])
rate, audioFile = wavfile.read(audioPath)
audioFile = audioFile.copy()
print(audioFile)
print(audioFile.shape)

[[  0   0]
 [  0   0]
 [  0   0]
 ...
 [-13 -14]
 [-13 -14]
 [-13 -14]]
(481489, 2)


In [3]:
chunkSize = len(msgInBinary)
nChunks = int(np.ceil(audioFile.shape[0] / chunkSize))
print(nChunks, chunkSize)

5016 96


In [4]:
if len(audioFile.shape) == 1:
  audioFile.resize(nChunks * chunkSize, refcheck=False)
  audioFile = audioFile[np.newaxis]
else:
  audioFile.resize((nChunks * chunkSize, audioFile.shape[1]), refcheck=False)
  audioFile = audioFile.T
chunks = audioFile[0].reshape((nChunks, chunkSize))

In [5]:
chunks = np.fft.fft(chunks)
phases = np.angle(chunks)
magnitudes = np.abs(chunks)
phaseDiff = np.diff(phases, axis=0)
nmsgInBinary = [-1 if i == '0' else 1 for i in msgInBinary]
nmsgInBinary = np.array(nmsgInBinary)
nmsgInBinary = nmsgInBinary * -np.pi / 2

In [6]:
print(chunks[0][1])
print(phases[0][1])
print(np.angle(chunks[0][1]))
print(magnitudes[0][1])
print(magnitudes[0][1] * np.exp(1j * phases[0][1]))

# change phase of chunks[0][1]


(-0.2073448001688729-1.0480101015611956j)
-1.7661200552592655
-1.7661200552592655
1.068324407252487
(-0.207344800168873-1.0480101015611956j)


In [7]:
phases[0] = nmsgInBinary
print(phases[0])
# for i in range(1, len(phases)):
#   phases[i] = phases[i-1] + phaseDiff[i-1]

[ 1.57079633 -1.57079633 -1.57079633  1.57079633 -1.57079633  1.57079633
  1.57079633  1.57079633  1.57079633 -1.57079633 -1.57079633  1.57079633
  1.57079633 -1.57079633  1.57079633 -1.57079633  1.57079633 -1.57079633
 -1.57079633  1.57079633 -1.57079633 -1.57079633  1.57079633  1.57079633
  1.57079633 -1.57079633 -1.57079633  1.57079633 -1.57079633 -1.57079633
  1.57079633  1.57079633  1.57079633 -1.57079633 -1.57079633  1.57079633
 -1.57079633 -1.57079633 -1.57079633 -1.57079633  1.57079633  1.57079633
 -1.57079633  1.57079633  1.57079633  1.57079633  1.57079633  1.57079633
  1.57079633 -1.57079633 -1.57079633 -1.57079633  1.57079633 -1.57079633
 -1.57079633 -1.57079633  1.57079633 -1.57079633 -1.57079633  1.57079633
 -1.57079633 -1.57079633 -1.57079633 -1.57079633  1.57079633 -1.57079633
 -1.57079633 -1.57079633  1.57079633  1.57079633 -1.57079633  1.57079633
  1.57079633 -1.57079633 -1.57079633  1.57079633 -1.57079633 -1.57079633
  1.57079633  1.57079633  1.57079633 -1.57079633 -1

In [12]:
print(np.exp(1j))

(0.5403023058681398+0.8414709848078965j)


In [8]:
chunks = magnitudes * np.exp(1j * phases)
chunks = np.fft.ifft(chunks).real
audioFile[0] = chunks.ravel().astype(np.int16)
wavfile.write("output.wav", rate, audioFile.T)

In [9]:
rate2, en_file2 = wavfile.read("output.wav")
chunkSize2 = len(msgInBinary)

In [10]:
code2 = np.fft.fft(en_file2)
phases2 = np.angle(code2)
phases2 = phases2.T
phases2 = phases2[0, :chunkSize]
phases2
# codeBin2 = (phases2 < 0).astype(np.int16)
# codeInIntCode2 = codeBin2.reshape((-1, 8)).dot(1 << np.arange(8 - 1, -1, -1))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
phases2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
print("".join([str(i) for i in codeBin2]))
print(msgInBinary)

001011010011011011010100101101101100100101010010010110101011011001001001011010100100100110100101
011010000110010101101100011011000110111100100000011101110110111101110010011011000110010000100100


In [ ]:
# plot graph of phases
import matplotlib.pyplot as plt
#  width of plot
plt.figure(figsize=(20, 10))
# plot phases2 over it
plt.plot(phases2)
plt.plot(phases[0])
plt.show()

In [16]:
print("".join(np.char.mod("%c", codeInIntCode2)).replace("~", ""))

-6Ô¶ÉRZ¶IjI¥
